# Caret-Separated Text

Caret-Separated Text (or CST) is a key-value pair format represented by numbers as keys and the value is the string enclosed between carets (^) that contains the translation. Any text which is not enclosed with carets is considered a comment and ignored.

## CST.NET

CST.NET uses .NET's built-in indexing extension function to accomplish locating of each respective key. As a consequence, it does not matter what you use for keys. I added an additional normalization to the pipeline that converts the document's line endings to the system's, in order to prevent crashes.

In [1]:
using System.IO;
using System.Collections.Generic;
using System.Text.RegularExpressions;

In [1]:
public static class CaretSeparatedText
{
    const char CARET = '^';
    static readonly string _lf = "\u000A";
    static readonly string _cr = "\u000D";
    static readonly string _crlf = "\u000D\u000A";
    static readonly string _ls = "\u2028";

    /// <summary>
    /// Gets the value from the integer-based key.
    /// </summary>
    /// <returns>Returns the entry</returns>
    public static string Parse(string content, int key)
    {
        var entries = NormalizeEntries(content);
        return GetEntry(entries, key.ToString());
    }

    /// <summary>
    /// Gets the value from the string-based key.
    /// </summary>
    /// <returns>Returns the entry</returns>
    public static string Parse(string content, string key)
    {
        var entries = NormalizeEntries(content);
        return GetEntry(entries, key);
    }

    /// <summary>
    /// Replaces the document's line endings with the native system line endings.
    /// </summary>
    /// <remarks>This stage ensures there are no crashes during parsing.</remarks>
    static IEnumerable<string> NormalizeEntries(string content)
    {
        if (!content.Contains(Environment.NewLine))
        {
            if (content.Contains(_lf))
                content = content.Replace(_lf, Environment.NewLine);

            if (content.Contains(_cr))
                content = content.Replace(_cr, Environment.NewLine);

            if (content.Contains(_crlf))
                content = content.Replace(_crlf, Environment.NewLine);

            if (content.Contains(_ls))
                content = content.Replace(_ls, Environment.NewLine);
        }

        var lines = content.Split(new[] { $"{CARET}{Environment.NewLine}" },
            StringSplitOptions.RemoveEmptyEntries);
        var entries = new List<string>();

        foreach (var line in lines)
        {
            // Skip comments
            if (line.StartsWith("//") || line.StartsWith("#") ||
                line.StartsWith("/*") || line.EndsWith("*/"))
                continue;

            entries.Add(line);
        }

        return entries;
    }

    static string GetEntry(IEnumerable<string> entries, string key)
    {
        // Search through list
        foreach (var entry in entries)
        {
            // If the line doesn't start with the key, keep searching.
            if (!entry.StartsWith(key))
                continue;

            // Locate index, trim carets and return translation.
            var startIndex = entry.IndexOf(CARET);
            var line = entry.Substring(startIndex);

            return line.TrimStart(CARET).TrimEnd(CARET);
        }

        return "***MISSING***";
    }
}

In [1]:
class ContentStrings
{
    string Language { get; set; } = "english";

    public string GetText(string table, int key) => GetText(table, key.ToString());

    public string GetText(string table, string key)
    {
        var baseDir = Path.Combine(Environment.CurrentDirectory, "data", "uitext", $"{Language}.dir");
        var files = Directory.GetFiles(baseDir);

        foreach (var file in files)
        {
            var id = Path.GetFileName(file);
            var second = id.IndexOf("_", 1);

            if (second == -1)
                continue;

            id = id.Substring(1, second - 1);

            if (id != table)
                continue;

            var content = File.ReadAllText(file);
            return CaretSeparatedText.Parse(content, key);
        }

        return "***MISSING***";
    }

    public static string CSTFile(string cst, string key)
    {
        var path = Path.Combine(AppContext.BaseDirectory, cst);
        var file = File.ReadAllText(path);

        return CaretSeparatedText.Parse(file, key);
    }
}

In [1]:
var english = new ContentStrings();
var v1Path = Path.Combine(Environment.CurrentDirectory, "data", "v1.cst");
var v1File = File.ReadAllText(v1Path);
var one = english.GetText("102", "Singleline");
/*var three = CaretSeparatedText.Parse(v1File, 3);
var four = CaretSeparatedText.Parse(v1File, 4); */
Console.WriteLine($"One:{Environment.NewLine}{one}");
/*Console.WriteLine($"Three:{Environment.NewLine}{three}");
Console.WriteLine($"Four:{Environment.NewLine}{four}"); */

In [1]:
var v2Path = Path.Combine(Environment.CurrentDirectory, "data", "v2.cst");
var v2File = File.ReadAllText(v2Path);
var singleLineV2 = CaretSeparatedText.Parse(v2File, "Singleline");
var multiLineV2 = CaretSeparatedText.Parse(v2File, "Multiline");
Console.WriteLine($"Single line v2:{Environment.NewLine}{singleLineV2}");
Console.WriteLine($"Multiline v2:{Environment.NewLine}{multiLineV2}");